In [2]:
#from _typeshed import NoneType
import logging
import sys
import os
from tkinter.messagebox import NO
from tokenize import Single
from types import FrameType
from typing import Final
from numpy.core.defchararray import lower
from numpy.core.numeric import NaN
# import qwt
from My_classes import *
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtCore import QThread, QTimer, pyqtSignal, QMutex, QMutexLocker
from PyQt5.QtGui import QImage, QPixmap
from QPlainTextEditLogger import QPlainTextEditLogger
from mainWindow import Ui_MainWindow
import queue
import copy
from scipy.io import savemat
from AHATNDITracker import *
from ThreadFactory import *
import pyigtl
from scipy.io import savemat,loadmat
from scipy import optimize
import json
from sksurgerynditracker.nditracker import NDITracker

In [13]:
igtlClient=pyigtl.OpenIGTLinkClient("127.0.0.1", 18944)

In [11]:
_tran=np.diag([1,1,1,1])
_tran

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]])

In [29]:

_tran=np.diag([1,1,1,1])
_tran[1,3]=30
mess=pyigtl.TransformMessage(_tran,device_name="Tool_1")
igtlClient.send_message(mess)

True

In [47]:
def ToolDefLossFunction(targetpose,args):
            AllData=args
            Loss=np.mean(np.sqrt(np.sum((AllData-targetpose)**2,1)))
            return Loss

_upperlim=256*15
_downlim=256*1.5
_minSize=10
_maxSize=300
_distanceTolerance=12
_preretval=0
_prelabel=[]
_prestats=[]
_precentroids=[]
_prelabelcorr=[]
_intrin=loadmat('AHAT_Para_Python.mat')
mtx=_intrin['Mtx']
distcoef=_intrin['dist']

AllAHATData=np.load("Cache/OfflineVIdeoName.npy",allow_pickle=True)
Data_Length=len(AllAHATData)
xyzs=[]
for num_fr in range(Data_Length):
    Curr_fr=AllAHATData[num_fr]
    _Frame=Curr_fr.MatReflectivity
    _DepthFrame=Curr_fr.MatDepth
    # _im=_Frame
    _im=copy.deepcopy(_Frame)
    _im[_im>=_upperlim]=_upperlim
    _im[_im<=_downlim]=_downlim
    _im=(_im-_downlim)*(256*256/_upperlim)/(_upperlim-_downlim)*255
    _mask=np.array(_im/256,dtype='uint8')
    _im[_mask==0]=0
    _displayTag=np.zeros([512,512])
    # retval,labels,stats,centroids=cv2.connectedComponentsWithStats(_mask,connectivity=8)
    retval,labels,stats,centroids,label_corr=DetectBalls(_mask,_minSize,_maxSize,_distanceTolerance,_preretval,[],[],_precentroids,_prelabelcorr)
    _preretval=retval
    _precentroids=centroids
    _prelabelcorr=label_corr
    _temp_xy = cv2.undistortPoints(centroids,mtx,distcoef)
    _xyd=[]
    _xyz=[]
    for j in range(centroids.shape[0]):
        _u=centroids[j,0]
        _v=centroids[j,1]
        _d=_DepthFrame[int(_v),int(_u)]
        _xyd.append([_temp_xy[j][0][0],_temp_xy[j][0][1],_d])
        _ori_xyz=[_temp_xy[j][0][0],_temp_xy[j][0][1],1]
        _l=np.sqrt(sum(np.array(_ori_xyz)**2))
        _real_xyz=_ori_xyz/_l*_d
        _lreal=np.sqrt(sum(np.array(_real_xyz)**2))
        _xyz.append(list(_real_xyz/_lreal*(_lreal+5.75)))
    xyzs.append(_xyz)
DataDefCollected=np.array(xyzs)
BaseFrame=DataDefCollected[0]
NumPoints=BaseFrame.shape[0]
assert NumPoints>=4 
DetectionTolerance=5
EffectivePoints=[]
for i in range(DataDefCollected.shape[0]):
    # 
    DataFrame=DataDefCollected[i]
    FrameDataWithRightSequence=[]
    for j in range(NumPoints):
        Dist=np.sum((DataFrame-BaseFrame[j])**2,1)
        if min(Dist)>=DetectionTolerance:
            continue
        FrameDataWithRightSequence.append(DataFrame[np.where(Dist==min(Dist))[0][0]])
    if not len(FrameDataWithRightSequence)==NumPoints:
        continue
    # reshape n*3->3n
    EffectivePoints.append(np.reshape(FrameDataWithRightSequence,[3*NumPoints]))
ToolDefFrames=np.array(EffectivePoints)
EffectiveFrameNum=ToolDefFrames.shape[0]
print("Effective Frame Num : "+str(EffectiveFrameNum))
MeanToolDefintion=np.mean(ToolDefFrames,0)
print(np.reshape(MeanToolDefintion,[4,3]))
Res=optimize.minimize(ToolDefLossFunction,MeanToolDefintion,args=(ToolDefFrames,))
FinalLoss=Res['fun']/np.sqrt(NumPoints)
ToolShape=np.reshape(Res['x'],[NumPoints,3])
ToolShape=ToolShape-np.mean(ToolShape,0)

Effective Frame Num : 293
[[-3.67684773e+01 -6.20147240e+01  5.43377121e+02]
 [ 1.43693985e-01 -2.75785411e+01  5.20371205e+02]
 [-8.41007179e+01 -2.44683831e+01  5.46004876e+02]
 [ 4.48670047e+01 -1.54733376e+01  5.03816456e+02]]


In [49]:
FinalLoss

0.5971314963332756

(300, 4, 3)

In [34]:
Data_Length

300